<a href="https://colab.research.google.com/github/componavt/topkar-space/blob/main/wd_compare/compare_two_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jellyfish
!pip install --upgrade google-api-python-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 40.0 MB/s eta 0:00:00
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.1.1
    Uninstalling google-auth-httplib2-0.1.1:
      Successfully uninstalled google-auth-httplib2-0.1.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0


In [ ]:
from itertools import chain
from difflib import SequenceMatcher
from ctypes import sizeof
import time
import jellyfish as jf
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
from googleapiclient.discovery import build
import pprint
import io
import csv


In [ ]:
#!wget https://github.com/componavt/topkar-space/raw/master/wd_compare/island_query.txt
!wget https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/island_query.txt
# cat island_query.txt

!wget https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/islands10.txt

!wget https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/lake_query.txt
!wget https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/lakes10.txt
!wget https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/island_q.txt
!wget https://raw.githubusercontent.com/componavt/topkar-space/main/key/winged-complex-421609-672350f9d66f.json

--2024-04-27 10:32:11--  https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/island_query.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2164 (2.1K) [text/plain]
Saving to: ‘island_query.txt.3’

island_query.txt.3  100%[===================>]   2.11K  --.-KB/s    in 0s      

2024-04-27 10:32:11 (23.6 MB/s) - ‘island_query.txt.3’ saved [2164/2164]

--2024-04-27 10:32:11--  https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/islands10.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [ ]:
def find_shared_elements(file1, file2, parametr):
  start_time = time.time()
  f2 = open(file1, 'r', encoding='utf-8')
  f1 = open(file2, 'r',encoding='utf-8')

  file1 = [line.strip().split('	') for line in f1]
  file2 = [line.strip().split('	') for line in f2]


  c,c2 = [],[] #лист для записи данных с Wikidata и Топкар соответственно
  for i in file2:
    for j in file1:
      if SequenceMatcher(None,i[1],j[1]).ratio() > parametr:
        c.append(i)
        c2.append(j)


  final = list(zip(c,c2))#лист с парами данных Wikidata/Топкар
  print(c)
  print(len(c))

  f1.close()
  f2.close()
  print("--- %s seconds ---" % (time.time() - start_time))
  return(final)

In [ ]:
from ctypes import sizeof
def find_shared_elements_jf(file1, file2,parametr):
  start_time = time.time()
  f2 = open(file1, 'r', encoding='utf-8')
  f1 = open(file2, 'r',encoding='utf-8')

  file1 = [line.strip().split('	') for line in f1]
  file2 = [line.strip().split('	') for line in f2]


  c,c2,c3 = [],[],[]#лист для записи данных с Wikidata и Топкар соответственно
  for i in file2:
    for j in file1:
      lev = jf.levenshtein_distance(i[1],j[1])
      if lev < parametr:
        c.append(i)
        c2.append(j)
        c3.append(lev)

  final = list(zip(c,c2,c3))#лист с парами данных Wikidata/Топкар
  print(final)

  f1.close()
  f2.close()
  print("--- %s seconds ---" % (time.time() - start_time))
  return(final,c,c2)

Функция открывает два файла (file1 и file2) для чтения и считывает их содержимое. Затем она разделяет каждую строку на элементы, используя разделитель '  ', и сохраняет результат в виде списка списков.

Далее функция проходит через каждую пару элементов из file2 и file1 и вычисляет расстояние Левенштейна между вторыми элементами каждой пары (i[1] и j[1]). Если расстояние меньше заданного значения parametr, то эта пара добавляется в соответствующие списки c, c2 и c3.

Затем функция создает список final, объединяя списки c, c2 и c3 в кортежи. После этого она выводит final на экран.



In [ ]:
# c1 = find_shared_elements('island_q.txt', 'islands10.txt', 0.9)
resived,c,c2 = find_shared_elements_jf('island_q.txt', 'islands10.txt', 2)

nodes = c+c2 #Все обекты из Wikidata и Топкар
# print(len(nodes))
sorted_nodes = [list(x) for x in set(tuple(x) for x in nodes)] #Удаляем дубликаты
# print(len(sorted_nodes))
def write_nodes_to_csv(c1, filename):
    with open(filename, 'w', newline='',encoding='utf-8') as csvfile:
        header = ['Id', 'Label', 'Whereform']
        # header = ['Id', 'Label']
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(header)
        for row in c1:
            print(row)
            if row[0][0] == 'h':
              row.append("wiki")
              csvwriter.writerow(row)
            else:
              row.append("topkar")
              csvwriter.writerow(row)

def write_edges_to_csv(c1, filename):
    with open(filename, 'w', newline='',encoding='utf-8') as csvfile:
        header = ['Source', 'Target', 'Weight']
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(header)
        for row in c1:
          # if row[2]>0:
            print(row)
            out = [row[0][0],row[1][0], row[2]+1]
            csvwriter.writerow(out)


filename_nodes = 'output_n.csv'
filename_edges = 'output_e.csv'
write_nodes_to_csv(sorted_nodes, filename_nodes) #создаём файл с вершинами гафа
write_edges_to_csv(resived, filename_edges) #создаём файл с рёбрами графа


[(['http://www.wikidata.org/entity/Q4212426', 'Канзанаволок'], ['39093', 'Канзанаволок']), (['http://www.wikidata.org/entity/Q4230532', 'Кондостров'], ['13958', 'Кондостров']), (['http://www.wikidata.org/entity/Q4230532', 'Кондостров'], ['16102', 'Кондостров']), (['http://www.wikidata.org/entity/Q4230532', 'Кондостров'], ['22970', 'Колдостров']), (['http://www.wikidata.org/entity/Q4234186', 'Корписари'], ['55705', 'Корбисари']), (['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['450', 'Лапостров']), (['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['6459', 'Лехостров']), (['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['7983', 'Лапостров']), (['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['14445', 'Лехостров']), (['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['15388', 'Липостров']), (['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['17279', 'Лехостров']), (['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['20470'

 Вызывается функция find_shared_elements_jf с указанными аргументами имён файлов и параметра

 Создается список sorted_nodes, в котором удаляются дубликаты объектов.

 Определяются две функции: write_nodes_to_csv и write_edges_to_csv, которые будут использоваться для записи данных в CSV-файлы.

  Функция write_nodes_to_csv записывает данные л вершинах графа из списка c1 в файл с именем filename в формате CSV, добавляя информацию о типе объекта ("wiki" или "topkar") в последний столбец в зависимости от значения первого элемента каждой строки.

  Функция write_edges_to_csv записывает данные о рёбрах графа из списка c1 в файл с именем filename в формате CSV.

  Наконец, вызываются функции write_nodes_to_csv и write_edges_to_csv, чтобы создать файлы с вершинами и ребрами графа

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = '/content/winged-complex-421609-672350f9d66f.json'
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=credentials)


folder_id = '1N5OznaW-E1wzuLAQXkAmtwuCXZEgBRLo'
name = 'output_e.csv'
file_path = '/content/output_e.csv'
file_metadata = {
                'name': name,
                'parents': [folder_id]
            }
media = MediaFileUpload(file_path, resumable=True)
r = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

folder_id = '1N5OznaW-E1wzuLAQXkAmtwuCXZEgBRLo'
name = 'output_n.csv'
file_metadata = {
                'name': name,
                'parents': [folder_id]
            }
file_path = '/content/output_n.csv'
media = MediaFileUpload(file_path, resumable=True)
r = service.files().create(body=file_metadata, media_body=media, fields='id').execute()


pp.pprint(r)

{'id': '1iZVkhveRQa6Om3vW5rjbP8sb8vR8_w1B'}


Устанавливаем необходимые права доступа (SCOPES) и загружаем файл сервисного аккаунта (SERVICE_ACCOUNT_FILE).

Создаем учетные данные (credentials) на основе файла сервисного аккаунта и указанных прав доступа.

Определяем идентификатор папки (folder_id), имя файла (name), путь к файлу на локальной машине (file_path) и метаданные файла для первого файла.

Загружаем первый файл на Google Drive с помощью метода files().create() с указанием метаданных и содержимого файла.

Повторяем предыдущие два шага для второго файла